# QR Constraint Networks 101

This notebook provides a brief example of how a constraint network can be:
* represented in JSON
* read-in from the JSON file to create a Network object
* summarized
* propagated

The constraint algebra defined by the Region Connection Calculus 8 (RCC8) is used for this example.

See the [Wikipedia webpage for RCC8](https://en.wikipedia.org/wiki/Region_connection_calculus).

## Imports

In [1]:
import qualreas as qr
import os

## Path to Network & Algebra

Here an environment variable is used to provide a machine/OS-independent path to the top-level Qualitative Reasoning directory

In [2]:
path = os.path.join(os.getenv('PYPROJ'), 'qualreas')
print(path)

/Users/alfredreich/Documents/Github/repo/qualreas


Networks can be loaded from a custom JSON file.

In [3]:
rcc8_file = path + "/Networks/rcc8_example.json"
print(rcc8_file)

/Users/alfredreich/Documents/Github/repo/qualreas/Networks/rcc8_example.json


The algebra used by a network is only specified by file name in the network's JSON file, so we need to provide the path (directory) to the file.

In [4]:
alg_dir = path + "/Algebras/"
print(alg_dir)

/Users/alfredreich/Documents/Github/repo/qualreas/Algebras/


## Constraint Network in JSON Format

Here's what a network looks like in JSON format.

A node is represented as a list of two things:
* Node ID
* List of ontology classes the node belongs to

An edge is represented as a list of three things, representing a directed edge with a "label":
* Head Node ID
* Target Node ID
* Constraint

Two Things to Note:
1. The Wikipedia page on RCC8 represents disjunctions of constraints as sets, {DC, EC}, whereas QR represents this with the string "DC|EC".
1. Constraints can be abbreviated using a dictionary of abbreviations.  For example, below, the constraint "DC|EC" is abbreviated as "dec".
1. No constraints are given for the Road-to-Property1 or Road-to-Property2 edges.  The meaning then is that all RCC8 relations are possible for those two edges.  This can be seen in the first summary of the network farther below.

In [5]:
!cat {rcc8_file}

{
    "name": "RCC8 Example 1",
    "algebra": "RCC8_Algebra",
    "abbreviations": {"dec": "DC|EC"},
    "description": "See https://en.wikipedia.org/wiki/Region_connection_calculus#Examples",
    "nodes": [
        ["House1", ["Region"]],
        ["House2", ["Region"]],
        ["Property1", ["Region"]],
        ["Property2", ["Region"]],
        ["Road", ["Region"]]
    ],
    "edges": [
        ["House1", "House2", "DC"],
        ["House1", "Property1", "TPP|NTPP"],
        ["House1", "Property2", "dec"],
        ["House1", "Road", "EC"],
        ["House2", "Property1", "dec"],
        ["House2", "Property2", "NTPP"],
        ["House2", "Road", "EC"],
        ["Property1", "Property2", "dec"],
        ["Road", "Property1"],
        ["Road", "Property2"]
    ]
}

## Load the Constraint Network

In [6]:
rcc8_net = qr.Network(algebra_path=alg_dir, json_file_name=rcc8_file)

print(rcc8_net)

<Network--RCC8 Example 1--RCC8_Algebra>


## Summarize the Network

This network is an example on the Wikipedia page for RCC8 (see "description" field, above).

Here's a summary of the network just loaded in.

The format is:
* Network Name: Number of Nodes, Number of Edges
* Algebra Name
* Head Node ID 1:
    * => Tail Node ID 1: Constraint 1
    * => Tail Node ID 2: Constraint 2
    * ...
* and so on ...

In [7]:
rcc8_net.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC|EC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC|EC
    => Property2: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI
    => Property2: DC|EC|EQ|NTPP|NTPPI|PO|TPP|TPPI


## Perform Constraint Propagation

After propagation, note the change in the constraints between the Road and the two Properties.

In [8]:
rcc8_net.propagate()
rcc8_net.summary()


RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC
    => Property2: DC|EC
    => Road: EC|PO
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: PO|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: EC|PO
    => Property2: PO|TPP


## Other Things We Can Do With Networks

### Get Entity by Name

In [9]:
entity = rcc8_net.get_entity_by_name("House1")
entity

SpatialObject(['Region'] 'House1')

In [10]:
entity.name

'House1'

In [11]:
entity.classes

['Region']

### Get Edge by Head/Tail Names

In [12]:
edge = rcc8_net.get_edge_by_names("Property1", "Road")
edge

('Property1', 'Road', 'EC|PO')

In [13]:
edge[2]  # It's just a list, so use square brackets to obtain items in it

'EC|PO'

### Network to Dictionary

In [14]:
rcc8_net_dict = rcc8_net.to_dict()

rcc8_net_dict

{'name': 'RCC8 Example 1',
 'algebra': 'RCC8_Algebra',
 'description': 'See https://en.wikipedia.org/wiki/Region_connection_calculus#Examples',
 'nodes': [['House1', ['Region']],
  ['House2', ['Region']],
  ['Property1', ['Region']],
  ['Property2', ['Region']],
  ['Road', ['Region']]],
 'edges': [['House1', 'House2', 'DC'],
  ['House1', 'Property1', 'NTPP|TPP'],
  ['House1', 'Property2', 'DC|EC'],
  ['House1', 'Road', 'EC'],
  ['House2', 'Property1', 'DC'],
  ['House2', 'Property2', 'NTPP'],
  ['House2', 'Road', 'EC'],
  ['Property1', 'Property2', 'DC|EC'],
  ['Property1', 'Road', 'EC|PO'],
  ['Property2', 'Road', 'PO|TPPI']]}

### Dictionary to Network

In [15]:
rcc8_net_from_dict = qr.Network(algebra_path=alg_dir, network_dict=rcc8_net_dict)

print(rcc8_net_from_dict)

rcc8_net_from_dict.summary()

<Network--RCC8 Example 1--RCC8_Algebra>

RCC8 Example 1: 5 nodes, 25 edges
  Algebra: RCC8_Algebra
  House1:
    => House1: EQ
    => House2: DC
    => Property1: NTPP|TPP
    => Property2: DC|EC
    => Road: EC
  House2:
    => House2: EQ
    => House1: DC
    => Property1: DC
    => Property2: NTPP
    => Road: EC
  Property1:
    => Property1: EQ
    => House1: NTPPI|TPPI
    => House2: DC
    => Property2: DC|EC
    => Road: EC|PO
  Property2:
    => Property2: EQ
    => House1: DC|EC
    => House2: NTPPI
    => Property1: DC|EC
    => Road: PO|TPPI
  Road:
    => Road: EQ
    => House1: EC
    => House2: EC
    => Property1: EC|PO
    => Property2: PO|TPP


### Another Dictionary to Network Example

Here we'll use the <b>Extended_Linear_Interval_Algebra</b>, that integrates proper time intervals (Allen's algebra) with time points.

The relation, "PF", is "PointFinishes", "PS" is "PointStarts", and "PE" is "PointEquals".

For example,
* <i>MondayMidnight PF Monday</i>
* <i>MondayMidnight PS Tuesday</i>
* <i>MondayMidnight PE MondayMidnight</i>

In [22]:
allen_net_dict = {
    'name': 'Allen Algebra Network',
    'algebra': 'Extended_Linear_Interval_Algebra',
    'description': 'A network based on Allen\'s algebra of proper intervals',
    'nodes': [
        ['Monday', ['ProperInterval']],
        ['MondayMidnight', ['Point']],
        ['Tuesday', ['ProperInterval']],
        ['MondayPM', ['ProperInterval']]
    ],
    'edges': [
        ['Monday', 'Tuesday', 'M'],
        ['MondayMidnight', 'Monday', 'PF'],
        ['MondayPM', 'Monday', 'F']
    ]
}

In [23]:
allen_net = qr.Network(algebra_path=alg_dir, network_dict=allen_net_dict)

print(allen_net.description)

allen_net.summary()

A network based on Allen's algebra of proper intervals

Allen Algebra Network: 4 nodes, 10 edges
  Algebra: Extended_Linear_Interval_Algebra
  Monday:
    => Monday: E
    => Tuesday: M
    => MondayMidnight: PFI
    => MondayPM: FI
  Tuesday:
    => Tuesday: E
    => Monday: MI
  MondayMidnight:
    => MondayMidnight: PE
    => Monday: PF
  MondayPM:
    => MondayPM: E
    => Monday: F


In [24]:
allen_net.propagate()
allen_net.summary()


Allen Algebra Network: 4 nodes, 16 edges
  Algebra: Extended_Linear_Interval_Algebra
  Monday:
    => Monday: E
    => Tuesday: M
    => MondayMidnight: PFI
    => MondayPM: FI
  Tuesday:
    => Tuesday: E
    => Monday: MI
    => MondayMidnight: PSI
    => MondayPM: MI
  MondayMidnight:
    => MondayMidnight: PE
    => Monday: PF
    => Tuesday: PS
    => MondayPM: PF
  MondayPM:
    => MondayPM: E
    => Monday: F
    => Tuesday: M
    => MondayMidnight: PFI
